# Análisis OLAP

El almacén de datos Hidro-Meto contiene observaciones de precipitación y temperatura de algunas estaciones ubicadas en distintas regiones del país.

El archivo `dw_hidro_meteo.sql` contiene el modelo del almacén de datos y los datos para poblarlo.

**Actividades**
2. Materializar un cubo en MySQL
3. En MySQL diseñar y ejecutar operaciones OLAP.
4. Conectar desde Python (`mysql-connector-python`), extraer y construir un cubo (agregaciones multidimensionales).
5. En Python diseñar y ejecutar operaciones OLAP.


**Requisitos**:
- MySQL 8.x
- Python 3.9+
- Pandas
- mysql-connector-python


## 1. Construcción y exploración del cubo en MySQL

1. Crear el cubo (tabla)
2. Extraer los datos desde el DW y cargarlos en el cubo (tabla)
2. Explorar el cubo con operaciones OLAP

## 2. Extraer datos del cubo y exploración OLAP con Python

In [2]:
import src.dw_connection as dw

# Verificar conexión y versión de MySQL
cnx = dw.conexion(host='localhost', user='root', pwd='mysqlroot', dbname='dw_hidro_meteo')
cnx.close()

MySQL version: 8.1.0
Conexión establecida con exito!


In [4]:
import pandas as pd

cnx = dw.conexion(host='localhost', user='root', pwd='mysqlroot', dbname='dw_hidro_meteo')
query = "SELECT * FROM mi_cubo"

# Alternativa A) Crear create_engine con sqlalchemy
# df = pd.read_sql(query, cnx)

# Alternativa B) Directamente con mysql-connector
cursor = cnx.cursor()
cursor.execute(query)
results = cursor.fetchall()
columns = [field[0] for field in cursor.description]
cubo_df = pd.DataFrame(results, columns=columns)

cursor.close()
cnx.close()

cubo_df.head(10)

MySQL version: 8.1.0
Conexión establecida con exito!


,anio,mes,dia,region,estacion,precip_mm,tmax_c
0,2024,1,1,Arica y Parinacota,ST001,26.30,28.30
1,2024,1,1,Tarapacá,ST002,33.20,30.50
2,2024,1,2,Arica y Parinacota,ST001,21.40,29.00
3,2024,1,2,Tarapacá,ST002,46.90,18.50
4,2024,1,3,Arica y Parinacota,ST001,53.60,15.20
5,2024,1,3,Tarapacá,ST002,30.70,12.10
6,2024,1,4,Arica y Parinacota,ST001,3.70,16.20
7,2024,1,4,Tarapacá,ST002,54.10,18.80
8,2024,1,5,Arica y Parinacota,ST001,12.80,17.60
9,2024,1,5,Tarapacá,ST002,14.50,24.70


In [5]:
# Roll-up: precipitación total por estación
cubo_df.groupby("estacion")["precip_mm"].sum()

estacion
ST001    10899.10
ST002    10925.50
Name: precip_mm, dtype: object

In [6]:
# Drill-down: precipitación día a día para ST001
cubo_df[cubo_df["estacion"] == "ST001"][["dia", "precip_mm"]]

,dia,precip_mm
0,1,26.30
2,2,21.40
4,3,53.60
6,4,3.70
8,5,12.80
...,...,...
722,27,18.40
724,28,6.30
726,29,54.00
728,30,27.90


In [7]:
# Slice: datos solo de ST002
cubo_df[cubo_df["estacion"]=="ST002"]

,anio,mes,dia,region,estacion,precip_mm,tmax_c
1,2024,1,1,Tarapacá,ST002,33.20,30.50
3,2024,1,2,Tarapacá,ST002,46.90,18.50
5,2024,1,3,Tarapacá,ST002,30.70,12.10
7,2024,1,4,Tarapacá,ST002,54.10,18.80
9,2024,1,5,Tarapacá,ST002,14.50,24.70
...,...,...,...,...,...,...,...
723,2024,12,27,Tarapacá,ST002,20.40,21.60
725,2024,12,28,Tarapacá,ST002,15.40,34.40
727,2024,12,29,Tarapacá,ST002,54.70,33.20
729,2024,12,30,Tarapacá,ST002,29.00,29.60


In [8]:
# Dice: promedio mensual de temperaturas
cubo_df.groupby(by=["estacion", "mes"])["tmax_c"].mean()

estacion  mes
ST001     1      22.129032
          2      22.965517
          3      20.619355
          4          20.34
          5      24.790323
          6      21.783333
          7      20.558065
          8      21.932258
          9      22.506667
          10     23.474194
          11     23.413333
          12     23.245161
ST002     1      23.558065
          2      24.593103
          3      21.858065
          4          21.16
          5      22.306452
          6          22.46
          7       21.33871
          8      23.125806
          9          24.17
          10     21.774194
          11          21.9
          12      23.43871
Name: tmax_c, dtype: object

In [9]:
# Pivot: días como filas, estaciones como columnas
pivot = cubo_df.pivot_table(index="mes", columns="estacion", values="precip_mm", aggfunc="sum")
print(pivot.head(35))

estacion    ST001    ST002
mes                       
1          725.60  1079.60
2          909.90   969.90
3          987.50   981.90
4          871.00   829.20
5          937.90  1054.20
6          876.40   743.70
7          987.50   776.70
8          907.70  1005.50
9          973.00   857.60
10        1014.00   902.30
11         863.70   840.60
12         844.90   884.30


## 3. Resumen

DW vs. Cubo
- **DW (f_observacion + dimensiones)**: granularidad diaria por estación y variable.
- **Cubo (cubo_meteo)**: agregaciones por combinaciones de dimensiones. Responde más rápido para análisis recurrentes.

## 4. Actividades

El almacén de datos `dw_hidro_climatico` contiene observaciones diarias de precipitación y temperatura máxima.
Las dimensiones incluyen:
- Tiempo (día, mes, trimestre, año)
- Estación (código, nombre)
- Localización (latitud, longitud, comuna, región)
- Variable, precipitación (PRECIP) y temperatura máxima (TMAX)

Se dispone de datos para 3 años (2022–2024) y 8 estaciones distribuidas en distintas regiones de Chile.

El objetivo es construir un cubo OLAP que permita explorar y realizar un análisis de la información desde distintas perspectivas.

#### Ejercicio 1. Crear y poblar el cubo

1. Construir en MySQL el cubo `meteo`, agregando los datos mensuales por (año, mes, región, estación) con las siguientes medidas:
- `precipitacion_mm = SUM(PRECIP)`
- `temperatura_c = AVG(TMAX)`
- `dias_de_lluvia = COUNT(DIAS CON PRECIP > 0)`

2. Inserta en el cubo las agregaciones de todas las estaciones para el periodo comprendido entre 2022 y 2023.

3. Entregar un archivo SQL con,
- Sentencias SQL para crear del cubo (`CREATE TABLE`).
- Sentencias SQL para poblar el almacen de datos (`INSERT INTO`).
- Sentencias SQL para verificar que los datos fueron cargados correctamente (`SELECT`).

#### Ejercicio 2: Análisis OLAP

Para cada problema identificar la(s) operacion(es) OLAP que permite resolver cada pregunta. Justificar.

1.	Calcular la precipitación total anual por región y estación para cada año del periodo. Identificar la estación más lluviosa en cada año.
2. A partir de los totales anuales, desglose los valores de 2023 en meses para la Estación Santiago. Identifique el mes que tuvo más días de lluvia.
3. Filtre el cubo para analizar la región del Biobío. Compare la temperatura promedio entre los años comprendidos en el periodo e identifique si se presentan cambios significativos.
4. Extraiga un subcubo con los meses de junio, julio y agosto (invierno) para todas las estaciones. Además, calcule para cada estación la precipitación total en invierno, la temperatura promedio de invierno y los días de lluvia en invierno. Explique si existes alguna relación entre las estaciones más lluviosas y las más frías.
5. Construya una tabla en la cual, las filas correspondan a los meses del año 2023, las columnas correspondas a las estaciones y las celdas contengan las precipitaciones. Identifica los meses con contrastes marcados de lluvia entre estaciones.
6. Genere un indicador de intensidad de lluvia: `intensidad = precipitacion_mm / días_de_lluvia`. Calcule este valor para cada estación en 2024 e identifique las estaciones que presentan lluvias menos frecuentes pero más intensas.
7. Considere que el cubo se debe ampliar a 20 años y a más estaciones del territorio chileno. ¿Qué jerarquías temporales y espaciales serían útiles para el análisis? ¿Es conveniente mantener un cubo materializado o calcularlo a demanda? Justifique su respuesta.